# Lab 6 CNNS

### Eli Laird

## Introduction

It is estimated that 23,820 adults in the United States will be diagnosed with brain cancer in 2019. Brain and central nervous system cancer is the tenth leading cause of death for men and women in the United States. The 5-year survival rate men and women with brain cancer is estimated to be between 34 and 36 percent [1]. Early detection of brain tumors is critical in effectively treating patients with brain tumors. In this lab, we will investigate methods for detecting brain tumors using deep Convolutional Neural Networks and Magnetic Resonance Imaging (MRI).

## Business Understanding

The ability to detect and treat a low-grade glioma, today, before it progresses to Glioblastoma multiforme, i.e. brain cancer, can be the difference between life and death [2]. In the past decade, advancements in medical imaging have greatly improved the ability of doctors to diagnose brain tumors without exploratory surgery. Exploratory surgery is invasive and introduces many risks to the patient. Magnetic Resonance Imaging, MRI, and Computer-Assisted Tools, CAT scans, are some of the most popular medical imaging techniques used to diagnose brain tumors. These imaging techniques are also used in tumor segmentation and operation planning to provide precision and diagnosis constistency. Using Deep Learning to assist in the diagnosing process can improve the precision of a diagnosis by a considerable amount. In a study of brain tumor diagnosis by the Royal College of Physicians, 10% of patients, in a Scottish audit, had negative CT scans, i.e. a false negative claiming there were no signs of a brain tumor when in fact a brain tumor existed [3]. A negative scan can prove to be fatal as the undetected tumor grows to a higher grade state and will eventually spread to an unrecoverable state. Using Deep Learning to assist in brain tumor diagosis aims to alleviate this problem by combining the abilities of Convolutional Neural Networks and the domain knowledge of general practictioners to increase the reliability of diagnosing brain tumors. 

## Objective



## References


* [1] https://www.cancer.net/cancer-types/brain-tumor/statistics
* [2] https://weillcornellbrainandspine.org/early-detection-can-be-key-surviving-brain-tumor
* [3] https://jnnp.bmj.com/content/75/suppl_2/ii18